In [2]:
# %pip install pandas
# %pip install numpy
# %pip install matplotlib
# %pip install bs4
# %conda install -c scitools cartopy
# %pip install geoviews
# %conda install -c conda-forge geopandas
# %pip install geopandas

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup
# import geoviews as gv
# import geoviews.tile_sources as gvts
# from geoviews import dim, opts
# from cartopy import crs
# import mplleaflet
#gv.extension('bokeh')
import geopandas
import ipyleaflet
from ipyleaflet import basemaps, Map, Marker
import ipywidgets
import geocoder

%matplotlib notebook

In [3]:
# url of wikipedia page containing most populated east coast cities and their populations
response = requests.get(url="https://en.wikipedia.org/wiki/Eastern_United_States")
soup = BeautifulSoup(response.content, 'html.parser')

# gets all cities and populations from html page
_ = []
for ultag in soup.find_all('ul', {'class': 'gallery mw-gallery-traditional'}):
    for litag in ultag.find_all('li'):
        _.append(litag.text)

# new data frame
df = pd.DataFrame([])

# clean data frame containing cities and their populations
_ = pd.Series(_).str.strip().replace(r'\n','')
df['City'] = _.apply(lambda x: x.split('population: ')[0])
df['Population'] = _.apply(lambda x: x.split('population: ')[1])
df['Population Display'] = df['Population'].copy()
df['Population'] = df['Population'].apply(lambda x: x.replace(',','')).astype(float)

# Manually Add in State codes
df['State Code'] = ['NY','IL','PA','NC','FL','IN','OH','MI','DC','MA',
                    'TN','MD','TN','KY','WI','FL','VA','GA','NC','OH','FL',
                    'PA','OH','KY']

df

,City,Population,Population Display,State Code
0,New York City,8622698.0,"8,622,698",NY
1,Chicago,2695598.0,"2,695,598",IL
2,Philadelphia,1567827.0,"1,567,827",PA
3,Charlotte,859035.0,"859,035",NC
4,Jacksonville,821784.0,"821,784",FL
5,Indianapolis,820445.0,"820,445",IN
6,Columbus,787033.0,"787,033",OH
7,Detroit,713777.0,"713,777",MI
8,"Washington, D.C.",703608.0,"703,608",DC
9,Boston,667137.0,"667,137",MA


In [4]:
# # wiki page containing lat and long of cities in us
# response = requests.get(url="https://www.infoplease.com/us/geography/latitude-and-longitude-us-and-canadian-cities")
# soup2 = BeautifulSoup(response.content, 'html.parser')

# # get all elements in the table
# table = soup2.findAll('table')[0].findAll('td')

# # get all citites
# cities = pd.Series([table[i].text for i in range(0,len(table)) if i%6==0])
# # gets latitudes
# lat_left = pd.Series([table[i].text for i in range(0,len(table)) if (i%6==1)])
# lat_right = pd.Series([table[i].text for i in range(0,len(table)) if (i%6==2)])
# latitudes = lat_left + '.' + lat_right
# # gets longitudes
# long_left = pd.Series([table[i].text for i in range(0,len(table)) if (i%6==3)])
# long_right = pd.Series([table[i].text for i in range(0,len(table)) if (i%6==4)])
# longitudes = long_left + '.' + long_right

# # create map for referencing lats and longs
# lat_long_map = pd.DataFrame([])
# lat_long_map['City'] = cities
# lat_long_map['Latitude'] = latitudes
# lat_long_map['Longitude'] = longitudes

# # merge prep for both data frames
# lat_long_map['merge_city'] = lat_long_map['City'].copy().apply(lambda x: x.split(',')[0])
# df['merge_city'] = df['City'].copy().apply(lambda x: x.split(',')[0])
# # update original df with lats and longs
# df = pd.merge(df, lat_long_map, how='left', left_on='merge_city', right_on='merge_city')

# # Fills in New York and Lexington Informations
# df = df.drop([0, 23])
# data = [['New York City', 8622698.0, 'New York City', 'New York City', '40.47', '73.58'],
#        ['Lexington', 295803.0, 'Lexington', 'Lexington', '38.04', '84.50']] 
# nyc_lex = pd.DataFrame(data, columns = df.columns) 
# df = df.append(nyc_lex)
# df = df.sort_values(by='Population', ascending=False).reset_index().drop(['index'], axis=1)

# # convert long at lat to float
# df['Latitude'] = df['Latitude'].astype(float)
# df['Longitude'] = df['Longitude'].astype(float).apply(lambda x: x-(2*x))

# df

In [5]:
# get cities map data set for long and lat values
url = 'https://raw.githubusercontent.com/kelvins/US-Cities-Database/main/csv/us_cities.csv'
cities_map = pd.read_csv(url, error_bad_lines=False)

# merge prep
df['merge_city'] = df['City'].copy().apply(lambda x: x.split(',')[0])
df = pd.merge(df, cities_map, how='inner', left_on=['City','State Code'], right_on=['CITY','STATE_CODE'])

# Drop Unnecessary records after vieeing merged dataframe, also missing New York and Washington DC
df = df.drop([2,5,6,8,13,15,21,25,28])
data = [['New York City', 8622698.0, '8,622,698', 'NY', 'New York City', '1234', 'NY', 'New York', 'New York City', 'New York', 40.72596, -73.99834], 
        ['Washington', 703608.0, '703,608', 'DC', 'Washington', '1235', 'DC', 'Washington', 'Washington', 'Washington', 38.893311, -77.014647]] 
nyc_dc = pd.DataFrame(data, columns = df.columns) 
df = df.append(nyc_dc)
df = df.sort_values(by='Population',ascending=False)
df = df.reset_index().drop(['index'], axis=1)

df['LATITUDE'] = df['LATITUDE'].astype(float)
df['LONGITUDE'] = df['LONGITUDE'].astype(float)
df

,City,Population,Population Display,State Code,merge_city,ID,STATE_CODE,STATE_NAME,CITY,COUNTY,LATITUDE,LONGITUDE
0,New York City,8622698.0,"8,622,698",NY,New York City,1234,NY,New York,New York City,New York,40.725960,-73.998340
1,Chicago,2695598.0,"2,695,598",IL,Chicago,6350,IL,Illinois,Chicago,Cook,41.885847,-87.618123
2,Philadelphia,1567827.0,"1,567,827",PA,Philadelphia,22798,PA,Pennsylvania,Philadelphia,Delaware,39.864998,-75.275196
3,Charlotte,859035.0,"859,035",NC,Charlotte,15054,NC,North Carolina,Charlotte,Mecklenburg,35.226900,-80.843300
4,Jacksonville,821784.0,"821,784",FL,Jacksonville,3890,FL,Florida,Jacksonville,Duval,30.316300,-81.417500
5,Indianapolis,820445.0,"820,445",IN,Indianapolis,7725,IN,Indiana,Indianapolis,Marion,39.775006,-86.109348
6,Columbus,787033.0,"787,033",OH,Columbus,19714,OH,Ohio,Columbus,Franklin,40.105155,-83.010069
7,Detroit,713777.0,"713,777",MI,Detroit,11706,MI,Michigan,Detroit,Wayne,42.347429,-83.060398
8,Washington,703608.0,"703,608",DC,Washington,1235,DC,Washington,Washington,Washington,38.893311,-77.014647
9,Boston,667137.0,"667,137",MA,Boston,10123,MA,Massachusetts,Boston,Suffolk,42.357603,-71.068432


In [6]:
# Mapping Visualization

#map_points = gv.Points(df, ['LONGITUDE', 'LATITUDE'], ['Population'])
#gvts.CartoDark.options(width=900, height=800, xaxis=None, yaxis=None, show_grid=False)  * map_points

# plt.figure(figsize=(8,8))
# plt.scatter(df['LONGITUDE'].tolist(), df['LATITUDE'].tolist(), c='r', alpha=0.7, s=150)
# mplleaflet.display()

radio_button = ipywidgets.RadioButtons(options=['Positron', 'DarkMatter', 'WorldStreetMap', 'DeLorme', 
                                                'WorldTopoMap', 'NatGeoWorldMap', 'HikeBike'],
                                       value='WorldStreetMap', 
                                       description='Map Type:')

# determine outputs for each radio button selected
def toggle_maps(map):
    if map == 'Positron': 
        m = Map(center=(40.67, -73.94), zoom=5, basemap=basemaps.CartoDB.Positron)
    if map == 'DarkMatter': 
        m = Map(center=(40.67, -73.94), zoom=5, basemap=basemaps.CartoDB.DarkMatter)
    if map == 'WorldStreetMap': 
        m = Map(center=(40.67, -73.94), zoom=5, basemap=basemaps.Esri.WorldStreetMap)
    if map == 'DeLorme': 
        m = Map(center=(40.67, -73.94), zoom=5, basemap=basemaps.Esri.DeLorme)
    if map == 'WorldTopoMap': 
        m = Map(center=(40.67, -73.94), zoom=5, basemap=basemaps.Esri.WorldTopoMap)
    if map == 'NatGeoWorldMap': 
        m = Map(center=(40.67, -73.94), zoom=5, basemap=basemaps.Esri.NatGeoWorldMap)
    if map == 'HikeBike': 
        m = Map(center=(40.67, -73.94), zoom=5, basemap=basemaps.HikeBike.HikeBike)
        
    for index,row in df.iterrows():
        marker = Marker(location=(row['LATITUDE'], row['LONGITUDE']), title= 'City: {}\nPopulation: {}'.format(row['City'],row['Population Display']))
        m.add_layer(marker)
        
    display(m)
    
# allow interactivity
ipywidgets.interact(toggle_maps, map=radio_button)



interactive(children=(RadioButtons(description='Map Type:', index=2, options=('Positron', 'DarkMatter', 'World…

<function __main__.toggle_maps(map)>